<a href="https://colab.research.google.com/github/priyank96/idl-spring-22-project-deepar/blob/main/DL_project_basic_model_all_data_with_vol_varun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!git clone https://github.com/priyank96/idl-spring-22-project-deepar


fatal: destination path 'idl-spring-22-project-deepar' already exists and is not an empty directory.


In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import logging
from sklearn import preprocessing
import torch.optim as optim
from tqdm import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
!python3 /content/idl-spring-22-project-deepar/src/data/company_data.py

  1% 11/1151 [00:01<03:00,  6.33it/s]/content/idl-spring-22-project-deepar/src/data/company_data.py:83: RuntimeWarning: divide by zero encountered in log
  np.log(float(day_data["1. open"])),
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in subtract
  return (a - mns) / sstd
  7% 80/1151 [00:19<05:25,  3.29it/s]<W> No Rows Returned for DOCN. Not enough samples probably.
  7% 86/1151 [00:20<02:35,  6.86it/s]/content/idl-spring-22-project-deepar/src/data/company_data.py:83: RuntimeWarning: divide by zero encountered in log
  np.log(float(day_data["1. open"])),
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:2419: R

In [4]:
import pickle
with open('/content/idl-spring-22-project-deepar/data/index_to_company.pkl','rb') as f:
  index_to_company = pickle.load(f)

with open('/content/idl-spring-22-project-deepar/data/company_to_index.pkl','rb') as f:
  company_to_index = pickle.load(f)

In [5]:
print(index_to_company)
print(company_to_index)

{0: 'LTRY', 1: 'RELI', 2: 'TLRY', 3: 'APPS', 4: 'ASAN', 5: 'NET', 6: 'AVLR', 7: 'XLE', 8: 'CL', 9: 'YHOO', 10: 'ONE', 11: 'SEAC', 12: 'VRM', 13: 'LMND', 14: 'MP', 15: 'MA', 16: 'DVN', 17: 'VET', 18: 'VLO', 19: 'DLTR', 20: 'BHP', 21: 'BB', 22: 'HNT', 23: 'UAVS', 24: 'SONY', 25: 'RWBYF', 26: 'IVZ', 27: 'LUV', 28: 'NTNX', 29: 'AMZN', 30: 'WISH', 31: 'TQLB', 32: 'T', 33: 'GEVO', 34: 'STLA', 35: 'SLB', 36: 'LEN', 37: 'CC', 38: 'PHUN', 39: 'GOGO', 40: 'APA', 41: 'TLT', 42: 'RIG', 43: 'ADBE', 44: 'PENN', 45: 'ENJ', 46: 'JBHT', 47: 'ALK', 48: 'IDEX', 49: 'RWB', 50: 'CRKR', 51: 'VZ', 52: 'RGBPP', 53: 'TNDM', 54: 'PSLV', 55: 'MSTR', 56: 'DVAX', 57: 'ROKU', 58: 'RSX', 59: 'SNDL', 60: 'DDOG', 61: 'ABNB', 62: 'ISRG', 63: 'GP', 64: 'ATVK', 65: 'HOG', 66: 'NVDA', 67: 'MAG', 68: 'ABBV', 69: 'CLF', 70: 'SNPW', 71: 'GOOG', 72: 'LOVE', 73: 'ZGNX', 74: 'TZA', 75: 'ANET', 76: 'INO', 77: 'ARCB', 78: 'VUZI', 79: 'PXD', 80: 'RUN', 81: 'EVGO', 82: 'NTLA', 83: 'TER', 84: 'SMH', 85: 'MTN', 86: 'OPEN', 87: 'WM', 

In [6]:
DATA_PATH = '/content/idl-spring-22-project-deepar/data'

stock_inputs = np.load(DATA_PATH + '/stock_inputs.npy', allow_pickle=True)
stock_labels = np.load(DATA_PATH + '/stock_labels.npy', allow_pickle=True)

stock_test_inputs = np.load(DATA_PATH + '/stock_test_inputs.npy',allow_pickle=True)
stock_test_labels = np.load(DATA_PATH + '/stock_test_labels.npy',allow_pickle=True)

In [7]:
NUM_TRAIN_SAMPLES = 500000
NUM_TEST_SAMPLES  = 100000

stock_inputs_trimmed = stock_inputs[:NUM_TRAIN_SAMPLES,:,:]
stock_labels_trimmed = stock_labels[:NUM_TRAIN_SAMPLES,:]

stock_test_inputs_trimmed = stock_test_inputs[:NUM_TEST_SAMPLES,:,:]
stock_test_labels_trimmed = stock_test_labels[:NUM_TEST_SAMPLES,:]

In [8]:
print('stock_inputs_trimmed shape', stock_inputs_trimmed.shape)
print('stock_labels_trimmed shape', stock_labels_trimmed.shape)
print('stock_test_inputs_trimmed shape', stock_test_inputs_trimmed.shape)
print('stock_test_labels_trimmed shape', stock_test_labels_trimmed.shape)

stock_inputs_trimmed shape (500000, 192, 5)
stock_labels_trimmed shape (500000, 192)
stock_test_inputs_trimmed shape (100000, 192, 5)
stock_test_labels_trimmed shape (100000, 192)


In [9]:
# On the Filtered input, check the number of actual companies retained

train_comp_ids = set()
test_comp_ids  = set()

# Every Seq in the window will belong to the same company
# Hence seq_id = 0
# cov_id = -1 (last index)
seq_id = 0
cov_id = -1

for sample in range(0, NUM_TRAIN_SAMPLES):
  train_comp_ids.add(stock_inputs_trimmed[sample][seq_id][cov_id])

for sample in range(0, NUM_TEST_SAMPLES):
  test_comp_ids.add(stock_test_inputs_trimmed[sample][seq_id][cov_id])

In [10]:
print(sorted(train_comp_ids))
print(len(train_comp_ids))
print(sorted(test_comp_ids))
print(len(test_comp_ids))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158

In [33]:
params = {
    'num_classes': len(train_comp_ids),
    'embedding_dim':64,
    'cov_dim': 4,
    'lstm_hidden_dim': 128,
    'lstm_layers':3 ,
    'window_size':192,
    'batch_size': 96,
    'learning_rate': 1e-3,
    'epochs':100,
    'num_test_samples': NUM_TEST_SAMPLES,
    'num_train_samples': NUM_TRAIN_SAMPLES,
    'conditioning_period': 168,
    'prediction_period': 24
}

print(params)

{'num_classes': 233, 'embedding_dim': 64, 'cov_dim': 4, 'lstm_hidden_dim': 128, 'lstm_layers': 3, 'window_size': 192, 'batch_size': 96, 'learning_rate': 0.001, 'epochs': 100, 'num_test_samples': 100000, 'num_train_samples': 500000, 'conditioning_period': 168, 'prediction_period': 24}


In [12]:
import numpy as np
import torch
from torch.utils.data import Dataset, Sampler
from pathlib import Path
import sys

DATA_PATH = '/content/idl-spring-22-project-deepar/data'


class TrainDataset(Dataset):
    def __init__(self):
        self.data = stock_inputs_trimmed
        self.label = stock_labels_trimmed
        self.train_len = self.data.shape[0]


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):
        x1 = torch.from_numpy(self.data[index].astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))
        return x1, x2  

class TestDataset(Dataset):
    def __init__(self):
        self.data = stock_test_inputs_trimmed
        self.label = stock_test_labels_trimmed
        self.test_len = self.data.shape[0]
        
    def __len__(self):
        return self.test_len

    def __getitem__(self, index):
      x1 = torch.from_numpy(self.data[index].astype(np.float32))
      x2 = torch.from_numpy(self.label[index].astype(np.float32))

      return x1, x2

In [13]:
def unwindow_sequence(dataset):
  company_data = {}

  seq_id = 0
  company_index_cov_id = -1

  for cov_ip, cov_op in dataset:

    comp_index = cov_ip[seq_id][company_index_cov_id]
    company_name = index_to_company[comp_index.item()]

    # IP Shape (Time, Dim)
    # OP Shape (Time, 1) # One covariate (open price) per time instant
    if company_data.get(company_name, None) is None:
      company_data[company_name] = [cov_ip, cov_op]
      
    else:
      # windows are created with stride 1
      # so we should be appending only the last element in the sequence
      cov_ip_last = cov_ip[-1, :].unsqueeze(0)
      cov_op_last = cov_op[-1].unsqueeze(0)

      # 0 --> Covariates
      # 1 --> Outputs, aka stock price
      company_data[company_name][0] = torch.cat((company_data[company_name][0], cov_ip_last), dim=0)
      company_data[company_name][1] = torch.cat((company_data[company_name][1], cov_op_last), dim=0)

  return company_data

In [14]:
train_data = TrainDataset()
test_data  = TestDataset()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=params['batch_size'], shuffle=False)

train_data_unwindowed = unwindow_sequence(train_data)
test_data_unwindowed = unwindow_sequence(test_data)

In [15]:
import plotly.express as px

def plot_output(company_data, company_name):
  # 1st index is output (prediction)
  x=np.linspace(1, len(company_data[company_name][1]), num=len(company_data[company_name][1]))
  fig = px.line(x=x, y=company_data[company_name][1])
  fig.show()
  print("op", company_data[company_name][1])

def plot_input_covariate(company_data, company_name, covariate_index):
  # 0th index is input (the covariates)
  x=np.linspace(1, company_data[company_name][0].shape[0], num=len(company_data[company_name][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[company_name][0][:,covariate_index])
  fig.show()
  print("ip", company_data[company_name][0][:,covariate_index])

In [16]:
# SANITY Check One: Output Label (open price) is shifted by one element of input (the ip opening price)
plot_output(train_data_unwindowed, 'MCD')
plot_input_covariate(train_data_unwindowed, 'MCD', 0)

op tensor([-0.6790, -0.7051, -0.6279,  ...,  0.4282,  0.4346,  0.4358])


ip tensor([-0.6889, -0.6790, -0.7051,  ...,  0.4525,  0.4282,  0.4346])


In [17]:
plot_input_covariate(train_data_unwindowed, 'MCD', 0)  # Opening Price
plot_input_covariate(train_data_unwindowed, 'MCD', 1)  # Volume
plot_input_covariate(train_data_unwindowed, 'MCD', 2)  # Day
plot_input_covariate(train_data_unwindowed, 'MCD', 3)  # Month
plot_input_covariate(train_data_unwindowed, 'MCD', 4)  # Company ID

ip tensor([-0.6889, -0.6790, -0.7051,  ...,  0.4525,  0.4282,  0.4346])


ip tensor([-0.4795, -1.5838,  0.8233,  ...,  0.4954, -0.0370, -0.2441])


ip tensor([-1.6811, -1.5670, -1.4528,  ...,  1.5150, -1.6811, -1.5670])


ip tensor([ 1.2869,  1.2869,  1.2869,  ..., -0.4512, -0.1615, -0.1615])


ip tensor([155., 155., 155.,  ..., 155., 155., 155.])


In [18]:
# Sanity Check - Plot Windows

import plotly.express as px

def plot_output_windowed(company_data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(company_data[window_id][1]), num=len(company_data[window_id][1]))
  fig = px.line(x=x, y=company_data[window_id][1])
  fig.show()
  print("op", company_data[window_id][1])

def plot_input_covariate_windowed(company_data, window_id, covariate_index):
  # 0th index is input (the covariates)
  x=np.linspace(1, company_data[window_id][0].shape[0], num=len(company_data[window_id][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[window_id][0][:,covariate_index])
  fig.show()
  print("ip", company_data[window_id][0][:,covariate_index])

In [19]:
plot_output_windowed(test_data, 0)
plot_input_covariate_windowed(test_data, 0, 0)

op tensor([0.8988, 0.6912, 0.8988, 0.8148, 0.9591, 0.8196, 0.8465, 0.8114, 0.9247,
        0.8378, 0.8026, 0.7849, 0.7449, 0.8290, 0.9376, 0.7982, 0.6003, 0.5079,
        0.6957, 0.8114, 1.3751, 1.4421, 1.6040, 1.7501, 1.6682, 1.6078, 1.6757,
        1.5697, 1.4694, 1.6154, 1.7538, 1.5198, 1.8649, 2.5644, 1.4421, 1.3313,
        1.6229, 1.1855, 1.4107, 1.2183, 1.1484, 0.8070, 0.4564, 0.9676, 0.8988,
        0.9376, 1.0398, 1.1691, 1.2040, 1.1608, 0.8727, 0.9847, 0.8944, 0.9376,
        1.2912, 1.1896, 0.7494, 1.0187, 0.6686, 0.5682, 0.6957, 0.7271, 1.1896,
        1.1773, 1.0356, 1.0651, 1.2019, 1.3513, 1.0861, 1.1649, 1.1111, 0.8070,
        0.7449, 0.5405, 0.4299, 0.6912, 0.4611, 0.6957, 1.1194, 1.2015, 1.3632,
        1.3233, 1.3313, 1.2019, 1.1773, 1.0017, 0.8944, 0.8814, 0.4846, 0.5266,
        0.5405, 0.4799, 0.6049, 0.6414, 0.5219, 0.7316, 1.0861, 1.0567, 0.9591,
        0.9161, 0.9161, 0.8596, 0.9419, 0.9591, 0.9505, 0.9161, 0.9591, 0.9376,
        1.2101, 1.2101, 1.0735, 1.044

ip tensor([0.7938, 0.8988, 0.6912, 0.8988, 0.8148, 0.9591, 0.8196, 0.8465, 0.8114,
        0.9247, 0.8378, 0.8026, 0.7849, 0.7449, 0.8290, 0.9376, 0.7982, 0.6003,
        0.5079, 0.6957, 0.8114, 1.3751, 1.4421, 1.6040, 1.7501, 1.6682, 1.6078,
        1.6757, 1.5697, 1.4694, 1.6154, 1.7538, 1.5198, 1.8649, 2.5644, 1.4421,
        1.3313, 1.6229, 1.1855, 1.4107, 1.2183, 1.1484, 0.8070, 0.4564, 0.9676,
        0.8988, 0.9376, 1.0398, 1.1691, 1.2040, 1.1608, 0.8727, 0.9847, 0.8944,
        0.9376, 1.2912, 1.1896, 0.7494, 1.0187, 0.6686, 0.5682, 0.6957, 0.7271,
        1.1896, 1.1773, 1.0356, 1.0651, 1.2019, 1.3513, 1.0861, 1.1649, 1.1111,
        0.8070, 0.7449, 0.5405, 0.4299, 0.6912, 0.4611, 0.6957, 1.1194, 1.2015,
        1.3632, 1.3233, 1.3313, 1.2019, 1.1773, 1.0017, 0.8944, 0.8814, 0.4846,
        0.5266, 0.5405, 0.4799, 0.6049, 0.6414, 0.5219, 0.7316, 1.0861, 1.0567,
        0.9591, 0.9161, 0.9161, 0.8596, 0.9419, 0.9591, 0.9505, 0.9161, 0.9591,
        0.9376, 1.2101, 1.2101, 1.073

In [20]:
class Network(nn.Module):
    def __init__(self, params):
        '''
        We define a recurrent network that predicts the 
        future values of a time-dependent variable based on
        past inputs and covariates.
        '''
        super(Network, self).__init__()
        self.params = params
        self.embedding = nn.Embedding(params['num_classes'], params['embedding_dim'])

        self.lstm = nn.LSTM(input_size=params['cov_dim']+params['embedding_dim'],
                            hidden_size=params['lstm_hidden_dim'],
                            num_layers=params['lstm_layers'],
                            bias=True,
                            batch_first=True,
                          )

        self.distribution_mu = nn.Linear(params['lstm_hidden_dim'], 1)
        self.distribution_presigma = nn.Linear(params['lstm_hidden_dim'], 1)
        self.distribution_sigma = nn.Softplus()


    def forward(self, x, h0_c0=None):
        '''
        Predict mu and sigma of the distribution for z_t.
        '''
        cov = x[:, :, :-1]   # remove the company index from the inputs to get the covariates

        company_index = x[:, 0, -1].to(torch.int32)  # retrieve the company index from the covariates
        # print("company_index = ", company_index)
        # print("params['num_classes'] = ", params['num_classes'])
        # print("params['embedding_dim'] = ", params['embedding_dim'])
        onehot_embed = self.embedding(company_index)
        batch_size = cov.shape[0]
        seq_len = cov.shape[1]
        cov_dim = cov.shape[2]

        assert cov_dim == params['cov_dim']
        assert batch_size <= params['batch_size']

        ohe_embed_all_timestamps = onehot_embed.unsqueeze(1).repeat(1,seq_len,1)

        lstm_input = torch.cat(
            (cov,
             ohe_embed_all_timestamps
            ), dim=2
          )
        
        assert lstm_input.shape[0] <= params['batch_size']
        assert lstm_input.shape[1] == seq_len
        assert lstm_input.shape[2] == params['cov_dim'] + params['embedding_dim']

        out1, hn_cn = self.lstm(input=lstm_input, hx=h0_c0)
              
        out_mu = self.distribution_mu(out1)
        
        out_presigma = self.distribution_presigma(out1)
        out_sigma = self.distribution_sigma(out_presigma)

        return out_mu, out_sigma, hn_cn

In [21]:
model = Network(params)

In [22]:
for i, (ip_covariate, op_label) in enumerate(train_loader):
  out_mu, out_sigma, ht_ct = model.forward(x=ip_covariate)

  print('out mu shape', out_mu.shape)
  print('out_sigma shape', out_sigma.shape)
  print('ht shape', ht_ct[0].shape, ht_ct[1].shape)


  break

out mu shape torch.Size([96, 192, 1])
out_sigma shape torch.Size([96, 192, 1])
ht shape torch.Size([3, 96, 128]) torch.Size([3, 96, 128])


In [23]:
def loss_fn(mu,sigma,labels):
  mu = torch.squeeze(mu)
  sigma = torch.squeeze(sigma)
  total_likelihood = 0
  for i in range(mu.shape[0]):  # loop through batch
    for j in range(mu.shape[1]):# each batch  loop through time steps
      distribution = torch.distributions.normal.Normal(mu[i][j] , sigma[i][j]) # scaling the values by 10 to avoid small sigma values
      total_likelihood += distribution.log_prob(labels[i][j])
  return -1*total_likelihood/(mu.shape[0]*mu.shape[1])

In [24]:
def loss_fn_updated(mu,sigma,labels):

  batch,seq_len,mudim = mu.shape
  batch,seq_len,sigmadim = sigma.shape

  assert mudim==sigmadim

  mu = mu.view(batch*seq_len,mudim)
  sigma = sigma.view(batch*seq_len,sigmadim)
  labels = labels.view(batch*seq_len,1)

  gauss_prices = torch.distributions.Normal(mu,sigma)
  total_likelihood = -1 * torch.sum(gauss_prices.log_prob(labels))/(batch*seq_len)

  return total_likelihood

In [25]:
out_price_mu = torch.randn(4, 8, 1)
out_price_sigma = torch.rand(4, 8, 1)
op_label_price = torch.rand(4, 8, 1)

print("orginal loss fn",loss_fn(out_price_mu, out_price_sigma,op_label_price))
print("update loss fn",loss_fn_updated(out_price_mu, out_price_sigma,op_label_price))

orginal loss fn tensor([10.7390])
update loss fn tensor(10.7390)


In [26]:
def accuracy_RMSE_train(mu: torch.Tensor, labels: torch.Tensor):
  predictions = mu.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

In [27]:
def accuracy_RMSE_test(predictions: torch.Tensor, labels: torch.Tensor):
  predictions = predictions.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

In [28]:
conditionining_period = params['conditioning_period']
prediction_period     = params['prediction_period']

assert conditionining_period + prediction_period == params['window_size']

In [29]:
def validate(model):

  model.eval()
  model.cuda()
  total_RMSE = 0

  overall_mu = None
  overall_sigma = None
  overall_label = None

  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):
      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period]

      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period: ]

      # Step One - Forward Pass : Conditioning Period    
      mu, sigma, ht_ct = model(x=cond_ip, h0_c0=None)

      batch_mu = mu.squeeze()
      batch_sigma = sigma.squeeze()

      # Initialize pred_mu for the first time instance of the "prediction period"
      # from the value of the "predicted mu" from the last instance of the "conditioning period"
      pred_mu, pred_sigma = mu[:, -1, :].unsqueeze(1), sigma[:, -1, :].unsqueeze(1)

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
        pred_cov_ip[:, 0, 0] = pred_mu[:, 0, 0]
        
        pred_mu, pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)
        
        batch_mu = torch.cat((batch_mu,  pred_mu.squeeze(2)), dim=1)
        batch_sigma = torch.cat((batch_sigma, pred_sigma.squeeze(2)), dim=1)


      if overall_mu is None and overall_sigma is None:
        overall_mu = batch_mu
        overall_sigma = batch_sigma
        overall_label = op_label
      else:
        overall_mu = torch.cat((overall_mu,  batch_mu), dim=0)
        overall_sigma = torch.cat((overall_sigma, batch_sigma), dim=0)
        overall_label = torch.cat((overall_label, op_label), dim=0)
        
  rmse = accuracy_RMSE_test(overall_mu, overall_label)

  return rmse

In [30]:
def plot_output_data(data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(data[window_id]), num=len(data[window_id]))
  fig = px.line(x=x, y=data[window_id])
  fig.show()
  # print("op", data[window_id])

In [31]:
validate(model)

1.1750662908681864

In [32]:
for i, (ip_covariate, op_label) in enumerate(train_loader):
  print(ip_covariate.shape)
  break

torch.Size([96, 192, 5])


In [34]:
model_version='deepar_model_no_volume_v1.pt'
epochs = params['epochs']

best_dev_rmse = 10000

model = Network(params)
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
#scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, threshold=0.01, threshold_mode='rel', cooldown=2, min_lr=0, eps=1e-08, verbose=False)
scaler = torch.cuda.amp.GradScaler()
print(model)

for epoch in range(0, epochs):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()
    model.cuda()

    total_loss = 0


    for i, (ip_covariate, op_label) in enumerate(train_loader):
        optimizer.zero_grad()

        ip_covariate = ip_covariate.cuda()
        # print(ip_covariate.shape)
        op_label     = op_label.cuda()

        mu, sigma, ht_ct = model(x=ip_covariate, h0_c0=None)

        # print('mu shape', mu.shape, 'sigma shape', sigma.shape, 'op label', op_label.shape)
        loss = loss_fn_updated(mu, sigma, op_label)

        # print("loss", loss)

        total_loss += float(loss)
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update()
        #loss.backward()
        #optimizer.step()

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        batch_bar.update() 

    batch_bar.close() # You need this to close the tqdm bar
    val_rmse= validate(model=model)
    scheduler.step(val_rmse)

    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss/len(train_loader),
            },  f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
  

    if val_rmse < best_dev_rmse:
      best_dev_rmse = val_rmse
    print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
    print("rmse is ", val_rmse)

Network(
  (embedding): Embedding(233, 64)
  (lstm): LSTM(68, 128, num_layers=3, batch_first=True)
  (distribution_mu): Linear(in_features=128, out_features=1, bias=True)
  (distribution_presigma): Linear(in_features=128, out_features=1, bias=True)
  (distribution_sigma): Softplus(beta=1, threshold=20)
)


Epoch 1/100: Train Loss -1.4769, Learning Rate 0.0010
rmse is  0.20758951479475146


Epoch 2/100: Train Loss -1.7565, Learning Rate 0.0010
rmse is  0.2062828437124599


Epoch 3/100: Train Loss -1.8343, Learning Rate 0.0010
rmse is  0.20584888007674232


Epoch 4/100: Train Loss -1.9072, Learning Rate 0.0010
rmse is  0.20486900761781346


...

Epoch 40/100: Train Loss -2.9569, Learning Rate 0.0002
rmse is  0.21774968649955262


Epoch 41/100: Train Loss -2.9634, Learning Rate 0.0002
rmse is  0.21766066984041624


Epoch 42/100: Train Loss -2.9654, Learning Rate 0.0002
rmse is  0.21783246146481147


Train:  92%|█████████▏| 4802/5209 [02:42<00:13, 29.53it/s, loss=-3.0386, lr=0.0002]

KeyboardInterrupt: ignored

In [35]:
import torch.distributions
def get_pred_sigma_mu_labels(model):

  model.eval()
  model.cuda()


  total_RMSE = 0

  overall_mu = None
  overall_sigma = None
  overall_label = None

  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):
      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period]

      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period: ]

      # Step One - Forward Pass : Conditioning Period    
      mu, sigma, ht_ct = model(x=cond_ip, h0_c0=None)

      batch_mu = mu.squeeze()
      batch_sigma = sigma.squeeze()
      

      # Initialize pred_mu for the first time instance of the "prediction period"
      # from the value of the "predicted mu" from the last instance of the "conditioning period"
      pred_mu, pred_sigma = mu[:, -1, :].unsqueeze(1), sigma[:, -1, :].unsqueeze(1) #-> pred_mu [BxTx1]
      
      m = torch.distributions.Normal(pred_mu, pred_sigma)
      sample = m.sample()
      sample_pred = sample.squeeze()
      batch_sampled = mu.squeeze()
      batch_sampled[:,-1] = sample.squeeze()

      # print("batch mu initial shape", batch_mu.shape)
      # print("batch sampled initial shape", batch_sampled.shape)

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
        # pred_cov_ip[:, 0, 0] = pred_mu[:, 0, 0]
        pred_cov_ip[:, 0, 0] = sample_pred

        pred_mu, pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)
        
        batch_mu = torch.cat((batch_mu,  pred_mu.squeeze(2)), dim=1)

        m = torch.distributions.Normal(pred_mu, pred_sigma)
        sample = m.sample()
        sample_pred = sample.squeeze()
        batch_sampled = torch.cat((batch_sampled,  sample.squeeze(2)), dim=1)

        # print("batch mu updated shape", batch_mu.shape)
        # print("batch sampled updated shape", batch_sampled.shape)



        batch_sigma = torch.cat((batch_sigma, pred_sigma.squeeze(2)), dim=1)


      if overall_mu is None and overall_sigma is None:
        overall_samples = batch_sampled
        overall_mu = batch_mu
        overall_sigma = batch_sigma
        overall_label = op_label
      else:
        overall_mu = torch.cat((overall_mu,  batch_mu), dim=0)
        overall_samples = torch.cat((overall_samples,  batch_sampled), dim=0)
        overall_sigma = torch.cat((overall_sigma, batch_sigma), dim=0)
        overall_label = torch.cat((overall_label, op_label), dim=0)
  

    # print("Shape of batch_mu", batch_mu.shape)
    # print("Shape of batch_sampled", batch_sampled.shape)

  return overall_samples, overall_mu, overall_sigma, overall_label

In [38]:
import pandas as pd
import plotly.graph_objects as go
def plot_test_output_data(actual,predicted,sampled, sigma, window_id):
  print("Predicted = ", predicted)
  x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
  df = pd.DataFrame()
  df['actual'] = actual[window_id]
  df['predicted'] = predicted[window_id]
  #df['sampled']=sampled[window_id]
  df['predicted_upper'] = predicted[window_id] + 1*sigma[window_id]
  df['predcited_lower'] = predicted[window_id] - 1*sigma[window_id]
  # print("df predicted = ", df['predicted'])

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
  fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))
  #fig.add_traces(go.Scatter(x=x, y = df['sampled'], mode = 'lines', name = 'Sampled'))

  fig.add_traces(go.Scatter(
        name='Upper Bound',
        x=x.tolist(),
        y=df['predicted_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ))
  fig.add_traces(
    go.Scatter(
        name='Lower Bound',
        x=x.tolist(),
        y= df['predcited_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ))
  

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Normalized Stock Opening Price')
  fig.show()

In [43]:
# samples_no_volume, mu_no_volume, sigma_no_volume, label_no_volume =  get_pred_sigma_mu_labels(model)
plot_test_output_data(label_no_volume.cpu().numpy(),mu_no_volume.cpu().numpy(), samples_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 5000)
# plot_test_output_data(label_no_volume.cpu().numpy(),samples_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 100)

Predicted =  [[ 0.72022974  0.8166441   0.7110492  ... -0.04364764 -0.06235766
  -0.04259628]
 [ 0.8303866   0.7080757   0.7896477  ...  0.05308292  0.11199437
   0.02663649]
 [ 0.6489316   0.8625344   0.82129115 ...  0.4231566   0.3677934
   0.34471154]
 ...
 [ 1.3126864   1.3346024   1.3458214  ...  1.2536132   1.2788188
   1.2901396 ]
 [ 1.2299685   1.2920613   1.3085089  ...  1.3050926   1.3220484
   1.2794862 ]
 [ 1.3521777   1.3021811   1.310296   ...  1.2322633   1.236941
   1.238125  ]]


In [ ]:
print(mu_no_volume)

tensor([0.7409, 0.7343, 0.7428,  ..., 1.1823, 1.2476, 1.2537], device='cuda:0')
